In [ ]:
!pip install optuna

In [ ]:
!pip install catboost

In [ ]:
isGpu = False
if isGpu:
    !git clone --recursive https://github.com/Microsoft/LightGBM
    %cd /content/LightGBM/
    !mkdir build
    !cmake -DUSE_GPU=1 #avoid ..
    !make -j$(nproc)
    !sudo apt-get -y install python-pip
    !sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
    %cd /content/LightGBM/python-package
    !sudo python setup.py install --precompile

In [ ]:
from common.supervised_learning import function
from common import classification

In [ ]:
# feature_selection range 50-150
n_features_to_select = 200

# supervised learning
## binary classification

Setting

In [ ]:
# 出力を predict_proba にするか 
is_predict_proba = True
# set pipelines for different algorithms
pipelines = {}
pipelines = classification.setup_algorithm(pipelines, is_predict_proba)

from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
evaluation_list = {'AUC':'roc_auc',
                   'F1':'f1',
                   'Recall':'recall',
                   'Precision':'precision',
                   'Accuracy':'accuracy'}
evaluation_function_list = {'AUC':roc_auc_score,
                            'F1':f1_score,
                            'Recall':recall_score,
                            'Precision':precision_score,
                            'Accuracy':accuracy_score}
options_evaluation = ['Accuracy', 'AUC', 'F1', 'Recall', 'Precision']
options_algorithm = ['lightgbm', 'xgboost', 'catboost', 'rsvc', 'logistic', 'rf', 'gb', 'mlp']
# 
exception_algorithm_list = ['tree', 'rsvc', 'gb', 'logistic']
exception_algorithm_list = ['tree', 'knn', 'rsvc', 'gb', 'rsvc', 'logistic', 'catboost', 'xgboost']
exception_algorithm_list = ['tree', 'knn', 'rsvc', 'gb', 'rsvc', 'logistic']

for algorithm in exception_algorithm_list:
    if algorithm in pipelines:
        pipelines.pop(algorithm)

# Feature selection by RandomForestClassifier
feature_selection_rf_list = ['knn', 'logistic', 'rsvc', 'mlp']
feature_importances_algorithm_list = ['tree', 'rf', 'lightgbm', 'xgboost', 'catboost']
tuning_prarameter = {
        'rf':{'max_depth': [3, 5, 7],
              'min_samples_leaf': [1, 2, 4]}
}
tuning_prarameter_list = ['gb', 'lightgbm', 'rf']
tuning_prarameter_list = ['lightgbm']
tuning_prarameter_list = ['logistic']
tuning_prarameter_list = ['rf']

In [ ]:
# 表示オプションの変更
import pandas as pd
pd.set_option('display.max_rows', 100)
# カテゴリ変数をリストで設定
ohe_columns = [
               #'Name',
               'Sex',
               #'Ticket',
               'Embarked',
               #'Ticket_no',
               'honorific',
               #'Cabin_head',
               #'FamilySize_bin'
               #'Cabin'
            ]
# カテゴリ変数をobject型で読み込むための準備
my_dtype = {
            #'Name':object,
            'Sex':object,
            'Ticket':object,
            'Embarked':object,
            'Cabin':object,
            'Pclass':'float32',
            'Age':'float32',
            'SibSp':'float32',
            'Parch':'float32',
            'Fare':'float32',
            }
id_label = ['PassengerId']
target_label = ['Perished']
drop_columns = ['Name', 'Ticket', 'Cabin', 'SibSp', 'Parch']
#drop_columns = ['Name', 'Ticket', 'Cabin', 'SibSp', 'Parch', 'Sex', 'Embarked']
log_columns = ['Pclass', 'Age', 'Fare']
train_file_name = input_data_dir + 'train.csv'
test_file_name = input_data_dir + 'test.csv'
model_columns_file_name = output_data_dir + 'model_columns.csv'
score_columns_file_name = output_data_dir + 'score_columns.csv'

# ファイル出力拡張子
file_extention = 'csv'

In [ ]:
is_header = True
# ホールドアウト有無
is_holdout = False
# k-fold 法を利用するか
is_k_fold = True
is_optuna = True
is_imputation = True
is_one_hot_encoding = True
is_feature_selection = False
isAdditionalCsv = False
isLog = False
is_category_encoded = True
is_multi_label_binarized = False
is_target_encoded = False
is_sklearn = False

In [ ]:
def input_train_file(filename, my_dtype):
    df = pd.read_csv(train_file_name, header=0, dtype=my_dtype)
    # データの形式に合わせて適時修正
    y = df.iloc[:,1]
    df = df.drop(id_label, axis=1)
    df = df.reset_index(drop=True)
    y.to_csv(output_data_dir + 'y.csv')
    class_mapping = {0:0, 1:1}
    y = y.map(class_mapping)
    X = df.drop(target_label, axis=1)
    X = X.reset_index(drop=True)
    X.to_csv(output_data_dir + 'X.csv')
    print('欠損個数（数値変数の欠損補完前）:input_train_file', X.isnull().sum().sum())
    print(X.isnull().sum())
    return X, y

In [ ]:
def input_test_file(filename, my_dtype, id_label):
    df_s = pd.read_csv(filename, header=0, dtype=my_dtype)
    X_s  = df_s.drop(id_label, axis=1)
    X_s = X_s.reset_index(drop=True)
    df_s = df_s.reset_index(drop=True)
    return df_s, X_s

In [ ]:
import pandas as pd
import datetime as dt
import re
import numpy as np
#def transform_data(X:pd.core.series.Series, y:pd.core.series.Series):
def transform_data(X:pd.core.series.Series):

    #X['Ticket'] = X['Ticket'].replace("\n","")
    #X['Cabin'] = X['Cabin'].replace("\n","")

    X['FamilySize'] = X['SibSp'] + X['Parch'] + 1
    X['IsAlone'] = 0
    X.loc[X['FamilySize'] == 1, 'IsAlone'] = 1
    '''
    X['FamilySize_bin'] = 'big'
    X.loc[X['FamilySize'] == 1, 'FamilySize_bin'] = 'alone'
    X.loc[(X['FamilySize'] >= 2) & (X['FamilySize']<=4), 'FamilySize_bin'] = 'small'
    X.loc[(X['FamilySize'] >= 5) & (X['FamilySize']<=7), 'FamilySize_bin'] = 'mediam'
    X = X.drop('FamilySize', axis=1)
    X = X.reset_index(drop=True)
    '''
    X['Ticket_len'] = X['Ticket'].apply(lambda x: len(x))
    #display(X['Ticket_len'].value_counts())
    '''
    X['Prefix'] = X['Name'].str.extract('([A-Za-z]+)[.]', expand=False)
    X['Prefix'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare', inplace=False)
    X['Prefix'].replace('Mlle', 'Miss', inplace=False)
    X['Prefix'].replace(['Ms', 'Mme'], 'Mrs', inplace=False)
    display(X['Prefix'].value_counts())
    prefix_mapping = {'Mr':1, 'Miss':2, 'Mrs':3, 'Master':4, 'Rare':5}
    X['Prefix'] = X['Prefix'].map(prefix_mapping)
    X['Prefix'] = X['Prefix'].fillna(0)
    display(X['Prefix'].value_counts())
    #X = function.one_hot_encoding(X, ['Prefix'])
    '''
    X['Ticket_no'] = X['Ticket'].str.extract('([0-9]+)').fillna(0)
    X['Ticket_no'] = X['Ticket_no'].astype(np.int64)
    name_df = X["Name"].str.split("[,.]", 2, expand=True)
    name_df.columns = ["family_name","honorific","name"]
    X["honorific"] = name_df["honorific"].str.strip()
    X.loc[~((X["honorific"] =="Mr") |
    (X["honorific"] =="Miss") |
    (X["honorific"] =="Mrs") |
    (X["honorific"] =="Master")), "honorific"] = "other"
    '''
    X['Cabin_head'] = X['Cabin'].apply(lambda x:str(x)[0])
    cabin_mapping = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'T':8}
    X['Cabin_head'] = X['Cabin_head'].map(cabin_mapping)
    X['Cabin_head'] = X['Cabin_head'].fillna(0, inplace=False)
    display(X['Cabin_head'].value_counts())
    #X = function.one_hot_encoding(X, ['Cabin_head'])
    '''
    if isLog:
        for column in log_columns:
            X['log_' + column] = np.log1p(X[column])
            X = X.drop(column, axis=1)
            X = X.reset_index(drop=True)

    if is_category_encoded:
        for column in ['Ticket_no']:
            #X['frequency_encode_' + column] = function.convert_to_frequency_encode(X, column)
            X['label_encode_' + column] = function.convert_to_label_encode(X, column)
            #X['count_encode_' + column] = function.convert_to_count_encode(X, column)
            #X['label_count_encode_' + column] = function.convert_to_label_count_encode(X, column)         
            if not is_target_encoded and not is_one_hot_encoding:
                for column in ['Ticket_no']:
                    X = X.drop(column, axis=1)
                    X = X.reset_index(drop=True)
    X = X.drop('Ticket_no', axis=1)
    X = X.reset_index(drop=True)
    # 不要カラム削除
    for column in drop_columns:
        X = X.drop(column, axis=1)
    X = X.reset_index(drop=True)
    return X

In [ ]:
input_evaluation = function.choice(options_evaluation)

RadioButtons(options=('Accuracy', 'AUC', 'F1', 'Recall', 'Precision'), value='Accuracy')

In [ ]:
X, y = input_train_file(train_file_name, my_dtype)

欠損個数（数値変数の欠損補完前）:input_train_file 866
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64


In [ ]:
pd.DataFrame(X).to_csv(output_data_dir + "X.csv", index=False)

In [ ]:
df_s, X_s = input_test_file(test_file_name, my_dtype, id_label)

In [ ]:
X = transform_data(X)
print('欠損個数（数値変数の欠損補完前）:input_train_file', X.isnull().sum().sum())
X_s = transform_data(X_s)

欠損個数（数値変数の欠損補完前）:input_train_file 179


In [ ]:
pd.DataFrame(X).to_csv(output_data_dir + "X_2.csv", index=False)

In [ ]:
if is_target_encoded:
    for column in ohe_columns:
        X[column] = X[column].fillna(-1)
        X_s[column] = X_s[column].fillna(-1)
    X, X_s = function.target_encode(X, y.values.ravel(), df_s, X_s, ohe_columns)
    for column in ohe_columns:
        if not is_one_hot_encoding:
            X = X.drop(column, axis=1)
            X = X.reset_index(drop=True)
            X_s = X_s.drop(column, axis=1)
            X_s = X_s.reset_index(drop=True)

In [ ]:
if is_one_hot_encoding:
    X_ohe = function.one_hot_encoding(X, ohe_columns)
elif not is_target_encoded and not is_category_encoded:
    for column in ohe_columns:
        X[column] = X[column].astype('category')
    X_ohe = X
else:
    X_ohe = X
    
#for column, is_empty in X_ohe.isnull().all(axis=0).iteritems():
#    if is_empty:
#        X_ohe = X_ohe.drop(column, axis=1)

print('欠損個数（数値変数の欠損補完前）', X_ohe.isnull().sum().sum())
print('')
print(X_ohe.isnull().sum())

欠損個数（数値変数の欠損補完前） 177

Pclass                      0
Age                       177
Fare                        0
FamilySize                  0
IsAlone                     0
Ticket_len                  0
label_encode_Ticket_no      0
Sex_female                  0
Sex_male                    0
Sex_nan                     0
Embarked_C                  0
Embarked_Q                  0
Embarked_S                  0
Embarked_nan                0
honorific_Master            0
honorific_Miss              0
honorific_Mr                0
honorific_Mrs               0
honorific_other             0
honorific_nan               0
dtype: int64


In [ ]:
if is_imputation:
    X_ohe, X_ohe_columns = function.imputation(output_data_dir, model_columns_file_name, X_ohe)
    print('欠損個数（数値変数の欠損補完前）', X_ohe.isnull().sum().sum())
    print('')
    print(X_ohe.isnull().sum())
else:
    X_ohe, X_ohe_columns = X_ohe, X_ohe.columns.values

欠損個数（数値変数の欠損補完前） 0

Pclass                    0
Age                       0
Fare                      0
FamilySize                0
IsAlone                   0
Ticket_len                0
label_encode_Ticket_no    0
Sex_female                0
Sex_male                  0
Sex_nan                   0
Embarked_C                0
Embarked_Q                0
Embarked_S                0
Embarked_nan              0
honorific_Master          0
honorific_Miss            0
honorific_Mr              0
honorific_Mrs             0
honorific_other           0
honorific_nan             0
dtype: int64


In [ ]:
pd.DataFrame(X_ohe).to_csv(output_data_dir + "X_ohe.csv", index=False)

Feature Selection

In [ ]:
import time
start_at = time.time()
#if is_feature_selection:
X_train, X_valid, y_train, y_valid = function.execute_feature_selection(output_data_dir,
                                                                    pipelines,
                                                                    feature_selection_rf_list,
                                                                    n_features_to_select, 
                                                                    X_ohe, y, X_ohe_columns,
                                                                    is_holdout,
                                                                    is_feature_selection)
#else:
#    X_train, X_valid, y_train, y_valid = X_ohe, X_ohe, y, y
end_at = time.time()
print("Elapsed time: {:.2f} [sec]".format(end_at - start_at))

ストリーミング出力は最後の 5000 行に切り捨てられました。
5006:	learn: 0.2865820	total: 8.37s	remaining: 8.34s
5007:	learn: 0.2865487	total: 8.37s	remaining: 8.34s
5008:	learn: 0.2865253	total: 8.37s	remaining: 8.34s
5009:	learn: 0.2865097	total: 8.37s	remaining: 8.34s
5010:	learn: 0.2864961	total: 8.37s	remaining: 8.34s
5011:	learn: 0.2864852	total: 8.37s	remaining: 8.33s
5012:	learn: 0.2864663	total: 8.38s	remaining: 8.33s
5013:	learn: 0.2864549	total: 8.38s	remaining: 8.33s
5014:	learn: 0.2864311	total: 8.38s	remaining: 8.33s
5015:	learn: 0.2864273	total: 8.38s	remaining: 8.33s
5016:	learn: 0.2864040	total: 8.38s	remaining: 8.32s
5017:	learn: 0.2863794	total: 8.38s	remaining: 8.32s
5018:	learn: 0.2863607	total: 8.38s	remaining: 8.32s
5019:	learn: 0.2863392	total: 8.39s	remaining: 8.32s
5020:	learn: 0.2863297	total: 8.39s	remaining: 8.32s
5021:	learn: 0.2863182	total: 8.39s	remaining: 8.31s
5022:	learn: 0.2863080	total: 8.39s	remaining: 8.31s
5023:	learn: 0.2862932	total: 8.39s	remaining: 8.31s
5024:	learn: 0

Modeling

In [ ]:
if is_one_hot_encoding:
    categorical_feature = []
elif is_target_encoded:
    categorical_feature = []
else:
    categorical_feature = ohe_columns
import time
start_at = time.time()
X_train, X_valid, y_train, y_valid = classification.train_model(output_data_dir, pipelines,
                                                                X_train, X_valid, y_train, y_valid,
                                                                input_evaluation.value, is_holdout,
                                                                is_optuna, is_sklearn, categorical_feature=categorical_feature)
end_at = time.time()
print("Elapsed time: {:.2f} [sec]".format(end_at - start_at))

[I 2020-10-25 07:59:00,494] A new study created in memory with name: no-name-45954e4d-9cd0-4b10-90c6-49e412d21165
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

lightgbm


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning:

Found `num_iterations` in params. Will use it instead of argument

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:123: UserWarning:

Found `early_stopping_rounds` in params. Will use it instead of argument

min_data_in_leaf, val_score: 0.004299: 100%|##########| 5/5 [00:04<00:00,  1.01it/s]


ストリーミング出力は最後の 5000 行に切り捨てられました。
5006:	learn: 0.2189275	total: 33.1s	remaining: 33s
5007:	learn: 0.2188914	total: 33.1s	remaining: 33s
5008:	learn: 0.2188746	total: 33.1s	remaining: 33s
5009:	learn: 0.2188401	total: 33.1s	remaining: 33s
5010:	learn: 0.2188107	total: 33.1s	remaining: 32.9s
5011:	learn: 0.2187935	total: 33.1s	remaining: 32.9s
5012:	learn: 0.2187735	total: 33.1s	remaining: 32.9s
5013:	learn: 0.2187520	total: 33.1s	remaining: 32.9s
5014:	learn: 0.2187273	total: 33.1s	remaining: 32.9s
5015:	learn: 0.2187014	total: 33.1s	remaining: 32.9s
5016:	learn: 0.2186741	total: 33.1s	remaining: 32.9s
5017:	learn: 0.2186535	total: 33.1s	remaining: 32.9s
5018:	learn: 0.2186167	total: 33.1s	remaining: 32.9s
5019:	learn: 0.2185967	total: 33.1s	remaining: 32.9s
5020:	learn: 0.2185540	total: 33.2s	remaining: 32.9s
5021:	learn: 0.2185387	total: 33.2s	remaining: 32.9s
5022:	learn: 0.2185010	total: 33.2s	remaining: 32.9s
5023:	learn: 0.2184765	total: 33.2s	remaining: 32.9s
5024:	learn: 0.2184462

In [ ]:
classification.display_evaluation(is_holdout, output_data_dir, pipelines, evaluation_function_list,
                                  input_evaluation, X_train, y_train, X_valid, y_valid, is_optuna,
                                  is_sklearn, is_predict_proba)

Accuracy
lightgbm
xgboost
xgboost
predict
catboost
catboost
predict
rf
rf
predict
mlp
mlp
predict
評価指標:Accuracy


,train
mlp,0.841751
xgboost,0.895623
catboost,0.922559
lightgbm,0.997755
rf,0.997755


In [ ]:
if is_optuna:
    import optuna.integration.lightgbm as lgb
    from sklearn.model_selection import StratifiedKFold
    dtrain = lgb.Dataset(X_train['lightgbm'], label=y_train['lightgbm'])
    params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
    }
    tuner = lgb.LightGBMTunerCV(
        params, dtrain, verbose_eval=100, early_stopping_rounds=100, folds=StratifiedKFold(n_splits=3)
    )
    tuner.run()
    print("Best score:", tuner.best_score)
    best_params = tuner.best_params
    print("Best params:", best_params)
    print("  Params: ")
    for key, value in best_params.items():
        print("    {}: {}".format(key, value))

[I 2020-10-25 08:01:25,837] A new study created in memory with name: no-name-f068a002-a233-43fa-8fc2-d4c0d3a3895c
feature_fraction, val_score: 0.423350:  14%|#4        | 1/7 [00:00<00:01,  3.64it/s]

[100]	cv_agg's binary_logloss: 0.511638 + 0.0545996


feature_fraction, val_score: 0.416554:  29%|##8       | 2/7 [00:00<00:01,  3.51it/s]

[100]	cv_agg's binary_logloss: 0.482933 + 0.0548539


feature_fraction, val_score: 0.416554:  43%|####2     | 3/7 [00:00<00:01,  3.60it/s]

...

[100]	cv_agg's binary_logloss: 0.416518 + 0.0410727
[100]	cv_agg's binary_logloss: 0.429664 + 0.0373361


min_data_in_leaf, val_score: 0.408241:  40%|####      | 2/5 [00:00<00:00,  7.18it/s]

[100]	cv_agg's binary_logloss: 0.434733 + 0.0337146
[200]	cv_agg's binary_logloss: 0.434382 + 0.0383565
[100]	cv_agg's binary_logloss: 0.414943 + 0.04919


min_data_in_leaf, val_score: 0.404321: 100%|##########| 5/5 [00:00<00:00,  7.31it/s]

[100]	cv_agg's binary_logloss: 0.419758 + 0.0402206
[100]	cv_agg's binary_logloss: 0.415545 + 0.0440917
Best score: 0.40432107328539607
Best params: {'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 0.010867777479007619, 'lambda_l2': 1.9559312649862672, 'num_leaves': 9, 'feature_fraction': 0.7, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 10}
  Params: 
    objective: binary
    metric: binary_logloss
    verbosity: -1
    boosting_type: gbdt
    feature_pre_filter: False
    lambda_l1: 0.010867777479007619
    lambda_l2: 1.9559312649862672
    num_leaves: 9
    feature_fraction: 0.7
    bagging_fraction: 1.0
    bagging_freq: 0
    min_child_samples: 10


In [ ]:
input_algorithm = function.choice(options_algorithm)

RadioButtons(options=('lightgbm', 'xgboost', 'catboost', 'rsvc', 'logistic', 'rf', 'gb', 'mlp'), value='lightg…

In [ ]:
import datetime
# 永続化したデータ読込し、 同じ評価指標ではtrain_modelは一度のみ実行で、かつ、選択されたモデルでスコアリングする
def main():
    if is_one_hot_encoding:
        X_ohe_s = function.one_hot_encoding(X_s, ohe_columns)
    elif not is_target_encoded and not is_category_encoded:
        for column in ohe_columns:
            X_s[column] = X_s[column].astype('category')
        X_ohe_s = X_s
    else:
        X_ohe_s = X_s

    #for column, is_empty in X_ohe_s.isnull().all(axis=0).iteritems():
    #    if is_empty:
    #        X_ohe_s = X_ohe_s.drop(column, axis=1)

    if is_imputation:
        X_ohe_s, X_ohe_columns = function.imputation(output_data_dir, model_columns_file_name, X_ohe_s)
    else:
        X_ohe_s, X_ohe_columns = X_ohe_s, X_ohe_s.columns.values

    algorithm_name = input_algorithm.value
    X_ohe_s.to_csv(output_data_dir + "X_ohe_s.csv", index=False, header=True)
    X_predicted = function.preprocessing(output_data_dir, model_columns_file_name, score_columns_file_name, algorithm_name, X_ohe,
                                         X_ohe_s, is_imputation, is_feature_selection)
    #print(X_predicted)
    predict = classification.scoring(output_data_dir, algorithm_name, X_predicted, input_evaluation,
                                     is_optuna, is_sklearn, is_predict_proba);
    #predict = [i if i > 0.01 else 0 for i in predict]
    function.output_file(output_data_dir, n_features_to_select, target_label, df_s, id_label,
                         predict, algorithm_name, file_extention, header=is_header)
    print(input_evaluation.value + ' selected')
    print(algorithm_name + ' selected')
    if algorithm_name in feature_importances_algorithm_list:
        if (is_optuna or not is_sklearn) and algorithm_name == 'lightgbm':
            ;
        else:
            feature_importances = pipelines[algorithm_name]['est'].feature_importances_
            feature_importances = pd.Series(feature_importances, index=X_predicted.columns.values.tolist())
            sorted_feature_importances = sorted(feature_importances.items(), key=lambda x:-x[1])
            sorted_feature_importances = pd.DataFrame(sorted_feature_importances)
            sorted_feature_importances.to_csv(output_data_dir + 'feature_importances_' + algorithm_name
                                              + '_' + datetime.datetime.now().strftime('%Y%m%d%H%M%S') + '.csv', index=False)
            display(sorted_feature_importances)

In [ ]:
if __name__ == '__main__':
    main()

モデルのみに存在する項目: set()
スコアのみに存在する項目: set()
lightgbm
Accuracy selected
lightgbm selected


In [ ]:
df_1 = pd.read_csv(output_data_dir + 'submission_lightgbm.csv', header=0, dtype=my_dtype)
df_2 = pd.read_csv(output_data_dir + 'submission_rf.csv', header=0, dtype=my_dtype)
df_3 = pd.read_csv(output_data_dir + 'submission_mlp.csv', header=0, dtype=my_dtype)
df_4 = pd.read_csv(output_data_dir + 'submission_catboost.csv', header=0, dtype=my_dtype)
df_5 = pd.read_csv(output_data_dir + 'submission_xgboost.csv', header=0, dtype=my_dtype)

In [ ]:
df_1['Perished'] = df_1['Perished'] + df_2['Perished'] + df_3['Perished'] + df_4['Perished'] + df_5['Perished']
df_1['Perished'] = (df_1['Perished'] >= 3).astype(int)
df_1.to_csv(output_data_dir + 'submission_ansemble' + '_' + datetime.datetime.now().strftime('%Y%m%d%H%M%S') + '.csv', index=False)

CV Score

In [ ]:
if is_k_fold:
    import time
    start_at = time.time()
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    classification.cross_validatior(kf, evaluation_list[input_evaluation.value], output_data_dir,
                                    n_features_to_select, pipelines, input_evaluation, X_ohe, y)
    end_at = time.time()
    print("Elapsed time: {:.2f} [sec]".format(end_at - start_at))